<h1 align="center">Round Robin</h1>

<h2>Changing to correct directory</h2>

In [1]:
import os
from pathlib import Path

os.chdir(Path.cwd().parent)
assert Path.cwd().name == "src" # make sure it is the correct folder

In [2]:
import pandas as pd
import random

from tournament_simulations.schedules import rename_teams_in_rounds, convert_list_of_rounds_to_dataframe
import tournament_simulations.schedules.round_robin as rr

<h2>Scheduling Algorithms</h2>

In [3]:
import tournament_simulations.schedules.algorithms as alg

In [4]:
# Circle-Method
random.seed(1)

# can also be accessed by "alg.name_to_scheduling_func["circle"](4)"
alg.CircleMethod.generate_schedule(4)

[((0, 3), (1, 2)), ((0, 2), (3, 1)), ((0, 1), (2, 3))]

<h2>Randomization</h2>

In [5]:
import tournament_simulations.schedules.randomize as rand

simple_schedule = alg.CircleMethod.generate_schedule(10)

In [6]:
randomizer = rand.RandomizeSchedule(simple_schedule)

# read method documentation for all available options
randomizer.randomize("all")

[((0, 1), (5, 7), (9, 3), (4, 6), (8, 2)),
 ((8, 1), (6, 9), (3, 2), (5, 0), (7, 4)),
 ((5, 8), (9, 0), (6, 3), (7, 2), (1, 4)),
 ((6, 7), (8, 4), (0, 3), (1, 2), (9, 5)),
 ((3, 1), (8, 7), (2, 5), (9, 4), (6, 0)),
 ((3, 5), (9, 8), (7, 0), (4, 2), (1, 6)),
 ((4, 5), (7, 3), (0, 2), (9, 1), (8, 6)),
 ((0, 4), (7, 9), (2, 6), (5, 1), (3, 8)),
 ((6, 5), (8, 0), (9, 2), (3, 4), (1, 7))]

<h2>Single Round-Robin</h2>

> All matches in a round have the same "date number"

In [7]:
random.seed(1)
single_rr = rr.SingleRoundRobin.from_num_teams(num_teams=5)

random.seed(1)
# in this case teams are randomized
random_single_rr = rr.SingleRoundRobin.from_num_teams(num_teams=5)

print(single_rr.schedule, end="\n\n")
print(random_single_rr.schedule)

[((0, 3), (1, 2)), ((4, 2), (0, 1)), ((3, 1), (4, 0)), ((2, 0), (3, 4)), ((1, 4), (2, 3))]

[((0, 3), (1, 2)), ((4, 2), (0, 1)), ((3, 1), (4, 0)), ((2, 0), (3, 4)), ((1, 4), (2, 3))]


In [8]:
random.seed(1)
list(single_rr.get_full_schedule(2))

[((1, 4), (3, 0)),
 ((0, 4), (2, 3)),
 ((4, 2), (0, 1)),
 ((3, 1), (0, 2)),
 ((3, 4), (1, 2)),
 ((4, 1), (3, 0)),
 ((3, 2), (0, 4)),
 ((0, 1), (4, 2)),
 ((1, 2), (4, 3)),
 ((0, 2), (3, 1))]

In [9]:
random.seed(1)
teams = ["zero", "one", "two", "three", "four"]
schedule = list(single_rr.get_full_schedule(2))
list(rename_teams_in_rounds(schedule, teams))

[(('one', 'four'), ('three', 'zero')),
 (('zero', 'four'), ('two', 'three')),
 (('four', 'two'), ('zero', 'one')),
 (('three', 'one'), ('zero', 'two')),
 (('three', 'four'), ('one', 'two')),
 (('four', 'one'), ('three', 'zero')),
 (('three', 'two'), ('zero', 'four')),
 (('zero', 'one'), ('four', 'two')),
 (('one', 'two'), ('four', 'three')),
 (('zero', 'two'), ('three', 'one'))]

In [10]:
original = convert_list_of_rounds_to_dataframe(single_rr.schedule, "tournament")

random.seed(1)
schedule = list(single_rr.get_full_schedule(1, to_randomize=None))
not_randomized = convert_list_of_rounds_to_dataframe(schedule, "tournament") 

random.seed(1)
schedule = list(single_rr.get_full_schedule(1))
randomized = convert_list_of_rounds_to_dataframe(schedule, "tournament") 

# Notice that non-randomized has the same ordering as the original one
pd.concat([original, not_randomized, randomized], axis=1, keys=["original", "not random", "random"])

original      not random      random     
                           home away       home away   home away
id         date number                                          
tournament 0                  0    3          0    3      1    4
           0                  1    2          1    2      3    0
           1                  4    2          4    2      0    4
           1                  0    1          0    1      2    3
           2                  3    1          3    1      4    2
           2                  4    0          4    0      0    1
           3                  2    0          2    0      3    1
           3                  3    4          3    4      0    2
           4                  1    4          1    4      3    4
           4                  2    3          2    3      1    2

In [11]:
random.seed(1)

# Arbitrary scheduling function
named_single_rr = rr.SingleRoundRobin.from_team_names(
    team_names=["a", "b", "c"],
    scheduling_func=lambda num_teams: [((i, i), (i + 1, i + 1)) for i in range(num_teams - 1)]
)

# in this one the order for all schedules is the same
print(list(named_single_rr.get_full_schedule(2, to_randomize=None)), end="\n\n")

# in this one the order for all schedules is fully randomized
print(list(named_single_rr.get_full_schedule(2)))

[(('a', 'a'), ('b', 'b')), (('b', 'b'), ('c', 'c')), (('a', 'a'), ('b', 'b')), (('b', 'b'), ('c', 'c'))]

[(('b', 'b'), ('a', 'a')), (('c', 'c'), ('a', 'a')), (('a', 'a'), ('b', 'b')), (('c', 'c'), ('a', 'a'))]


<h2>Double Round-Robin</h2>

> All matches in a round have the same "date number"

In [12]:
random.seed(1)
double_rr = rr.DoubleRoundRobin.from_num_teams(num_teams=4)

random.seed(1)
# in this case teams are randomized
random_double_rr = rr.DoubleRoundRobin.from_num_teams(num_teams=4)

# Second portion always have (home, away) flipped when compared to the first one
print(double_rr.first_schedule)
print(double_rr.second_schedule, end="\n\n")
print(random_double_rr.first_schedule)
print(random_double_rr.second_schedule)

[((0, 3), (1, 2)), ((0, 2), (3, 1)), ((0, 1), (2, 3))]
[((3, 0), (2, 1)), ((2, 0), (1, 3)), ((1, 0), (3, 2))]

[((0, 3), (1, 2)), ((0, 2), (3, 1)), ((0, 1), (2, 3))]
[((3, 0), (2, 1)), ((2, 0), (1, 3)), ((1, 0), (3, 2))]


In [13]:
random.seed(1)
# Everything is randomized here
list(double_rr.get_full_schedule(2))

[((1, 2), (0, 3)),
 ((2, 3), (0, 1)),
 ((2, 0), (3, 1)),
 ((2, 1), (3, 0)),
 ((3, 2), (1, 0)),
 ((0, 2), (1, 3)),
 ((2, 3), (0, 1)),
 ((3, 0), (1, 2)),
 ((3, 1), (2, 0)),
 ((3, 2), (1, 0)),
 ((0, 3), (2, 1)),
 ((1, 3), (0, 2))]

In [14]:
random.seed(2)

original = convert_list_of_rounds_to_dataframe(
    double_rr.first_schedule + double_rr.second_schedule, "tournament"
)
non_random = convert_list_of_rounds_to_dataframe(
    double_rr.get_full_schedule(1, to_randomize_first=None, to_randomize_second=None), "tournament"
) 
flipped = convert_list_of_rounds_to_dataframe(
    double_rr.get_full_schedule(1, to_randomize_first=None, to_randomize_second="flipped"), "tournament"
)
reversed = convert_list_of_rounds_to_dataframe(
    double_rr.get_full_schedule(1, to_randomize_first=None, to_randomize_second="reversed"), "tournament"
) 
second_random = convert_list_of_rounds_to_dataframe(
    double_rr.get_full_schedule(1, to_randomize_first=None, to_randomize_second="all"), "tournament"
) 
all_random = convert_list_of_rounds_to_dataframe(double_rr.get_full_schedule(1, "all", "all"), "tournament") 

# 1) No random is just a copy of the original (if there were more schedule it would just be contatenations of it)
# 2) When only first portion is randomized, the second portion can be symmetric to it
# 3) When second portion (date numbers [3, 4, 5]) is random, it doesn't have the same ordering as the first one
dataframe = pd.concat(
    [original, non_random, flipped, reversed, second_random, all_random], 
    axis=1, 
    keys=["original", "no random", "second portion flipped", "second portion reversed", "second portion random", "both random"]
)
round_mapping = {i: value for i, value in enumerate(["first"] * 3 + ["second"] * 3)}
new_index_level = dataframe.index.get_level_values("date number").map(round_mapping).rename("round")
dataframe.set_index(new_index_level, append=True)

original      no random       \
                                  home away      home away   
id         date number round                                 
tournament 0           first         0    3         0    3   
                       first         1    2         1    2   
           1           first         0    2         0    2   
                       first         3    1         3    1   
           2           first         0    1         0    1   
                       first         2    3         2    3   
           3           second        3    0         3    0   
                       second        2    1         2    1   
           4           second        2    0         2    0   
                       second        1    3         1    3   
           5           second        1    0         1    0   
                       second        3    2         3    2   

                              second portion flipped       \
                                                home away   
id         date number round                                
tournament 0           first                       0    3   
                       first                       1    2   
           1           first                       0    2   
                       first                       3    1   
           2           first                       0    1   
                       first                       2    3   
           3           second                      3    0   
                       second                      2    1   
           4           second                      2    0   
                       second                      1    3   
           5           second                      1    0   
                       second                      3    2   

                              second portion reversed       \
                                                 home away   
id         date number round                                 
tournament 0           first                        0    3   
                       first                        1    2   
           1           first                        0    2   
                       first                        3    1   
           2           first                        0    1   
                       first                        2    3   
           3           second                       3    2   
                       second                       1    0   
           4           second                       1    3   
                       second                       2    0   
           5           second                       2    1   
                       second                       3    0   

                              second portion random      both random       
                                               home away        home away  
id         date number round                                               
tournament 0           first                      0    3           0    3  
                       first                      1    2           2    1  
           1           first                      0    2           1    3  
                       first                      3    1           2    0  
           2           first                      0    1           0    1  
                       first                      2    3           3    2  
           3           second                     1    2           0    3  
                       second                     0    3           2    1  
           4           second                     2    3           3    2  
                       second                     0    1           0    1  
           5           second                     0    2           3    1  
                       second                     1    3           2    0

In [15]:
random.seed(1)

named_double_rr = rr.DoubleRoundRobin.from_team_names(team_names=["a", "b", "c", "d"])

# In this case you can see that the first portion for both schedules is the same
# first portion first schedule  -> rounds 1, 2, 3
# first portion second schedule -> rounds 7, 8, 9
rounds = list(named_double_rr.get_full_schedule(2, to_randomize_first=None))
rounds[:3] == rounds[6: 9]

True